<a href="https://colab.research.google.com/github/nickselv/Bayesian_Statistics_Project/blob/main/MODELLO_FINALE_NO_COEFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELLO CON BETA INDIP DA i

In [ ]:
# Dati di esempio: numero di eventi in diverse aree geografiche e tempi
data = {
    'N': data_stan.shape[0],  # Numero totale di osservazioni (spaziali-temporali)
    'T': len(data_stan['data'].unique()),  # Numero di periodi temporali unici (304)
    'I': len(data_stan['id_varco'].unique()),  # Numero di aree geografiche (40)
    'count': data_stan.groupby(['id_varco', 'data'])['numero_transiti'].sum().unstack().values.tolist(),    # Conteggi degli ingressi
    'time': time_matrix,  # Giorno dell'anno dalle date (la formula strana serve per avere valori da 1 a 304)
    'area': data_stan['id_varco'].values,        # Identificatori delle aree geografiche
    'precipitazioni': data_stan['precipitazioni'].values,    # Precipitazioni
    'av_temp': data_stan['av_temp'].values,              # Temperatura media
    'Xt': data_stan[['data', 'precipitazioni','av_temp','estate','inverno']].drop_duplicates().reset_index()[['precipitazioni', 'av_temp','estate','inverno']].transpose(),     #data_stan[['precipitazioni','av_temp']].values.transpose(),
    #'Xi': data_stan[['nome_varco','mezzi']].drop_duplicates().reset_index()[['mezzi']].transpose(),
    #'sc': covariance_matrix,     # mettere qua la matrice varianza e covarianza calcolata fuori da stan
    'fixed_w1':  2*math.pi/7,
    'fixed_w2': 2*math.pi/90,
    'output': output,
    'mu_c': media,
    'output_flat': np.zeros(I*T, dtype=int),
    'coeff': coeff
}

# Definizione del modello in Stan
stan_code = """
functions {
  // Definizione della funzione f(t)
  matrix f_matr(real a, real b, real c, real d, real w1, real w2, matrix t) {
    return a*sin(w1 * t) + b*cos(w1 * t) + c*sin(w2 * t) + d*cos(w2 * t);
  }



  matrix f_ma(real a, real b, real w, matrix A){
    int rows = rows(A);
    int cols = cols(A);
    matrix[rows, cols] result;

    for (i in 1:rows) {
      for (j in 1:cols) {
        result[i, j] = a * sin(w * A[i, j]) + b * cos(w * A[i, j]);
      }
    }
    return result;
  }


}


data {   //30
    int<lower=0> N;                       // Numero totale di osservazioni
    int<lower=0> T;                       // Numero di periodi temporali distinti   20
    int<lower=0> I;                       // Numero di aree geografiche distinti
    array[I,T] real count;             // Conteggi degli eventi
    matrix[I,T] time;        // Periodi temporali
    array[N] real area;        // Identificatori delle aree geografiche
    array[N] real precipitazioni;               // Precipitazioni
    array[N] real av_temp;                      // Temperatura media
    matrix[4,T] Xt;
    //matrix[1,I] Xi;
    row_vector[I] mu_c;
    row_vector[I] coeff;
    array[I * T] int<lower=0> output_flat;
    array[I,T] int<lower=0>  output;
    real fixed_w1;                    // Valore fisso per w
    real fixed_w2;


} //51

parameters {

    //real<lower=1e-4> x;
    row_vector[I] c;
    real a0;
    real b0;
    real a1;
    real b1;
    row_vector[4] beta;                    // Coefficienti per Xt


} //60



model {
    // Priors
    c ~ normal(mu_c, 10);                // c ~ normal(mu_c, 1);

    beta ~ normal(0, 1);
    a0 ~ normal(0, 1);
    b0 ~ normal(0, 1);
    a1 ~ normal(0, 1);
    b1 ~ normal(0, 1);



// Modello di Poisson spazio-temporale con f(t), ci, Xt e Xi    71
    // Calculate lambda values using STAN matrix operations
    matrix[I, T] lambda_matrix = f_matr(a0, b0, a1, b1, fixed_w1, fixed_w2, time) + rep_matrix(c, T)' +  rep_matrix(beta*Xt, I);

    // Calculate the Poisson log likelihood using STAN matrix operations
    row_vector[I*T] lambda_row = to_row_vector(lambda_matrix);

    // Calculate the Poisson log likelihood for the entire vectors
    output_flat ~ poisson(exp(lambda_row));

}

"""

# MODELLO CON BETA_i

In [ ]:
# Dati di esempio: numero di eventi in diverse aree geografiche e tempi
data = {
    'N': data_stan.shape[0],  # Numero totale di osservazioni (spaziali-temporali)
    'T': len(data_stan['data'].unique()),  # Numero di periodi temporali unici (304)
    'I': len(data_stan['id_varco'].unique()),  # Numero di aree geografiche (40)
    'count': data_stan.groupby(['id_varco', 'data'])['numero_transiti'].sum().unstack().values.tolist(),    # Conteggi degli ingressi
    'time': time_matrix,  # Giorno dell'anno dalle date (la formula strana serve per avere valori da 1 a 304)
    'area': data_stan['id_varco'].values,        # Identificatori delle aree geografiche
    #'longitude': data_stan['longitude'].values,  # Longitudine
    #'latitude': data_stan['latitude'].values,  # Latitudine
    'precipitazioni': data_stan['precipitazioni'].values,    # Precipitazioni
    'av_temp': data_stan['av_temp'].values,              # Temperatura media
    'Xt': data_stan[['data', 'precipitazioni','av_temp','estate','inverno']].drop_duplicates().reset_index()[['precipitazioni', 'av_temp','estate','inverno']].transpose(),     #data_stan[['precipitazioni','av_temp']].values.transpose(),
    #'Xi': data_stan[['nome_varco','nord','sud', 'est', 'ovest']].drop_duplicates().reset_index()[['nord','sud', 'est', 'ovest']].transpose(),
    #'Xi': data_stan[['nome_varco','longitude','latitude']].drop_duplicates().reset_index()[['longitude','latitude']].transpose(),
    #'sc': covariance_matrix,     # mettere qua la matrice varianza e covarianza calcolata fuori da stan
    'fixed_w1':  2*math.pi/7,
    'fixed_w2': 2*math.pi/90,
    'output': output,
    'mu_c': media,
    'output_flat': np.zeros(I*T, dtype=int),
    'coeff': coeff
}

# Definizione del modello in Stan
stan_code = """
functions {
  // Definizione della funzione f(t)
  real f(real a, real b, real w, real t) {
    return a * sin(w * t) + b * cos(w * t);
  }

 // Definizione della funzione f(t)
  matrix f_matr(real a, real b, real c, real d, real w1, real w2, matrix t) {
    return a*sin(w1 * t) + b*cos(w1 * t) + c*sin(w2 * t) + d*cos(w2 * t);
  }


  matrix f_ma(real a, real b, real w, matrix A){
    int rows = rows(A);
    int cols = cols(A);
    matrix[rows, cols] result;

    for (i in 1:rows) {
      for (j in 1:cols) {
        result[i, j] = a * sin(w * A[i, j]) + b * cos(w * A[i, j]);
      }
    }
    return result;
  }


}


data {   //30
    int<lower=0> N;                       // Numero totale di osservazioni
    int<lower=0> T;                       // Numero di periodi temporali distinti   20
    int<lower=0> I;                       // Numero di aree geografiche distinti
    array[I,T] real count;             // Conteggi degli eventi
    matrix[I,T] time;        // Periodi temporali
    array[N] real area;        // Identificatori delle aree geografiche
    //array[N] real longitude;               // Longitudine
    //array[N] real latitude;                     // Latitudine
    array[N] real precipitazioni;               // Precipitazioni
    array[N] real av_temp;                      // Temperatura media
    matrix[4,T] Xt;
    //matrix[4,I] Xi;
    row_vector[I] mu_c;
    row_vector[I] coeff;
    array[I * T] int<lower=0> output_flat;
    array[I,T] int<lower=0>  output;
    real fixed_w1;                    // Valore fisso per w
    real fixed_w2;                   // Valore fisso per w



} //51

parameters {
    matrix[I, 4] beta;                     // Coefficienti per Xt
    //row_vector[2] alpha;                   // Coefficienti per Xi
    real a0;
    real b0;
    real a1;
    real b1;
    row_vector[I] c;

    //real<lower=0> s;
} //60

model {
    // Priors
    c ~ normal(mu_c, 10);                // c ~ normal(mu_c, 1);


    for (k in 1:I) {
        for (j in 1:4) {
            beta[k, j] ~ normal(0, 1);  // Priori per beta sqrt(s1)
        }
    }

    a0 ~ normal(0, 1);
    b0 ~ normal(0, 1);
    a1 ~ normal(0, 1);
    b1 ~ normal(0, 1);


    // Modello di Poisson spazio-temporale con f(t), ci, Xt e Xi    61
    // Calculate lambda values using STAN matrix operations

     matrix[I, T] lambda_matrix = f_matr(a0, b0, a1, b1, fixed_w1, fixed_w2, time) + rep_matrix(c, T)' + beta*Xt;


    // Calculate the Poisson log likelihood using STAN matrix operations
    //row_vector[I*T] lambda_row = to_row_vector(lambda_matrix + rep_matrix(log(coeff), T)');
    row_vector[I*T] lambda_row = to_row_vector(lambda_matrix);

    // Calculate the Poisson log likelihood for the entire vectors
    output_flat ~ poisson(exp(lambda_row));




}



"""

# SOLO ESTATE-INVERNO

In [ ]:
# Dati di esempio: numero di eventi in diverse aree geografiche e tempi
data = {
    'N': data_stan.shape[0],  # Numero totale di osservazioni (spaziali-temporali)
    'T': len(data_stan['data'].unique()),  # Numero di periodi temporali unici (304)
    'I': len(data_stan['id_varco'].unique()),  # Numero di aree geografiche (40)
    'count': data_stan.groupby(['id_varco', 'data'])['numero_transiti'].sum().unstack().values.tolist(),    # Conteggi degli ingressi
    'time': time_matrix,  # Giorno dell'anno dalle date (la formula strana serve per avere valori da 1 a 304)
    'area': data_stan['id_varco'].values,        # Identificatori delle aree geografiche
    'precipitazioni': data_stan['precipitazioni'].values,    # Precipitazioni
    'av_temp': data_stan['av_temp'].values,              # Temperatura media
    #'Xt': data_stan[['data', 'precipitazioni','av_temp','estate','inverno']].drop_duplicates().reset_index()[['precipitazioni', 'av_temp','estate','inverno']].transpose(),     #data_stan[['precipitazioni','av_temp']].values.transpose(),
    'Xt': data_stan[['data', 'estate','inverno']].drop_duplicates().reset_index()[['estate', 'inverno']].transpose(),     #data_stan[['precipitazioni','av_temp']].values.transpose(),
    #'Xi': data_stan[['nome_varco','mezzi']].drop_duplicates().reset_index()[['mezzi']].transpose(),
    #'sc': covariance_matrix,     # mettere qua la matrice varianza e covarianza calcolata fuori da stan
    'fixed_w1':  2*math.pi/7,
    'fixed_w2': 2*math.pi/90,
    'output': output,
    'mu_c': media,
    'output_flat': np.zeros(I*T, dtype=int),
    'coeff': coeff
}

# Definizione del modello in Stan
stan_code = """
functions {
  // Definizione della funzione f(t)
  matrix f_matr(real a, real b, real c, real d, real w1, real w2, matrix t) {
    return a*sin(w1 * t) + b*cos(w1 * t) + c*sin(w2 * t) + d*cos(w2 * t);
  }



  matrix f_ma(real a, real b, real w, matrix A){
    int rows = rows(A);
    int cols = cols(A);
    matrix[rows, cols] result;

    for (i in 1:rows) {
      for (j in 1:cols) {
        result[i, j] = a * sin(w * A[i, j]) + b * cos(w * A[i, j]);
      }
    }
    return result;
  }


}


data {   //30
    int<lower=0> N;                       // Numero totale di osservazioni
    int<lower=0> T;                       // Numero di periodi temporali distinti   20
    int<lower=0> I;                       // Numero di aree geografiche distinti
    array[I,T] real count;             // Conteggi degli eventi
    matrix[I,T] time;        // Periodi temporali
    array[N] real area;        // Identificatori delle aree geografiche
    array[N] real precipitazioni;               // Precipitazioni
    array[N] real av_temp;                      // Temperatura media
    matrix[2,T] Xt;
    //matrix[1,I] Xi;
    row_vector[I] mu_c;
    row_vector[I] coeff;
    array[I * T] int<lower=0> output_flat;
    array[I,T] int<lower=0>  output;
    real fixed_w1;                    // Valore fisso per w
    real fixed_w2;


} //51

parameters {

    //real<lower=1e-4> x;
    row_vector[I] c;
    real a0;
    real b0;
    real a1;
    real b1;
    row_vector[2] beta;                    // Coefficienti per Xt


} //60



model {
    // Priors
    c ~ normal(mu_c, 10);                // c ~ normal(mu_c, 1);

    beta ~ normal(0, 2);
    a0 ~ normal(0, 1);
    b0 ~ normal(0, 1);
    a1 ~ normal(0, 1);
    b1 ~ normal(0, 1);



// Modello di Poisson spazio-temporale con f(t), ci, Xt e Xi    71
    // Calculate lambda values using STAN matrix operations
    matrix[I, T] lambda_matrix = f_matr(a0, b0, a1, b1, fixed_w1, fixed_w2, time) + rep_matrix(c, T)' +  rep_matrix(beta*Xt, I);

    // Calculate the Poisson log likelihood using STAN matrix operations
    row_vector[I*T] lambda_row = to_row_vector(lambda_matrix);

    // Calculate the Poisson log likelihood for the entire vectors
    output_flat ~ poisson(exp(lambda_row));

}


"""